In [113]:
import json
from tqdm import tqdm
from neo4j import GraphDatabase
from pymongo import MongoClient
import numpy as np
import pandas as pd

In [ ]:
uri = "bolt://localhost:7999"
username = "neo4j"
password = "password"

def create_neo4j_session(uri, username, password):
    try:
        driver = GraphDatabase.driver(uri, auth=(username, password))
        session = driver.session()
        return session
    except Exception as e:
        print(f"Failed to create Neo4j session: {e}")
        return None


## Deleting pre-existing nodes

In [ ]:
session = create_neo4j_session(uri, username, password)

# Execute a sample query
query = "MATCH (n) DETACH DELETE n"
result = session.run(query)

# Close the session and driver
session.close()

## Import Business Nodes

In [ ]:
def run_query(query):
    session = create_neo4j_session(uri, username, password)
    result = list(session.run(query))
    session.close()
    return result

In [ ]:
load_business_query = """CALL apoc.load.json("file:///active_business.json")
YIELD value
MERGE (b:Business {name: value.gmap_id})
SET b.name = value.name
SET b.latitude = value.latitude
SET b.longitude = value.longitude
WITH b, value
UNWIND value.category AS cat
MERGE (c:Category {name: toLower(cat)})
MERGE (b)-[:IS_IN]->(c);
"""

In [ ]:
spatial_query = """
CALL spatial.addPointLayer('geom');
"""

In [ ]:
# Create Business nodes and Category nodes
run_query(load_business_query)

# Create Spatial layer for indexing
run_query(spatial_query)

# Add Businesses to spatial index
run_query("""MERGE (n:Business) with n CALL spatial.addNode('geom', n) YIELD node RETURN node;""")


## Import Bus Stops

In [29]:
bus_import = """USING PERIODIC COMMIT 500 LOAD CSV WITH HEADERS FROM 'file:///SanDiegoStops.csv' AS line
MERGE (n:Stop {oid: line.OBJECTID})
set n.latitude = toFloat(line.stop_lat)
set n.longitude = toFloat(line.stop_lon)
set n.name = line.stop_name
set n.sid = line.stop_UID
"""

In [32]:
bus_line_import = """USING PERIODIC COMMIT 500 LOAD CSV WITH HEADERS FROM 'file:///StopsToRoutes.csv' AS line
MATCH (n:Stop {sid:line.stop_UID})
MERGE (l:Route {rid:line.route_id})
MERGE (n)-[:STOP_IN]->(l)"""

In [33]:
# import transit stops
run_query(bus_import)
# Add stops to spatial index
run_query("""MERGE (n:Stop) with n CALL spatial.addNode('geom', n) YIELD node RETURN node;""")
# Add transit route relationships to stops
run_query(bus_line_import)


[]

# Queries

In [117]:
def get_business_radius(lat, long, dist_mi, category=None):
    query = '''call spatial.withinDistance('geom', {latitude:''' +\
        str(lat) + ', longitude:' +\
        str(long) + '''}, ''' + str(dist_mi*1.61) +\
        ''') yield node as n, distance as d with n, d match(n:Business)'''
    if category is None:
        query += 'return n, d'
    else:
        query += '''-[:IS_IN]-(c:Category) where c.name contains ''' \
            + repr(category) + ''' return distinct n, c, d'''
    result = run_query(query)
    return result

In [105]:
def get_bus_stops(lat, long, dist_mi):
    query = '''call spatial.withinDistance('geom', {latitude:''' + str(lat) + ', longitude:' + str(long) + '''}, ''' + str(dist_mi*1.61) + ''') yield node as n, distance as d
    with n, d match (n:Stop)-[:STOP_IN]-(r) return r, n, d limit 10'''
    result = run_query(query)
    return result

In [66]:
def get_businesses_on_route(route_id, dist_mi):
    query = """match (n:Stop)-[:STOP_IN]-(r:Route) where r.rid = '""" + str(route_id)+ """' 
    with n call spatial.withinDistance('geom', {lat:n.latitude, lon:n.longitude}, """ + str(dist_mi*1.61) +""") yield node as b with b match (b:Business) return distinct b"""
    result = run_query(query)
    return result

In [133]:
def get_poi_bbox(bl_lat, bl_lon, tr_lat, tr_lon, poi='Both'):
    """
    poi can be Both, Business, Stops for the specified nodes
    """
    query = """call spatial.bbox('geom', {lat:""" + str(bl_lat) + ', lon: ' + str(bl_lon) + '}, {lat:' + str(tr_lat) + ', lon: ' + str(tr_lon) +"""}) yield node as n
    """
    if poi not in ['Both', 'Stops', 'Business']:
        print('Not Valid Point of Interest Type')
        return None
    if poi == 'Both':
        query += ' return distinct n'
    elif poi == 'Stops':
        query += ' match (n:Stops) return distinct n'
    elif poi == 'Business':
        query += ' match (n:Business) return distinct n'
    result = run_query(query)
    return result

In [130]:
result = get_business_radius(32.865028, -117.221756, 1)
for r in result:
    print(dict(dict(r)['n'])['name'] + ', Miles: ' + str(np.round(dict(r)['d']/1.6, 3)))

Vons, Miles: 0.198
Bristol Farms, Miles: 0.416
Costa Verde Center, Miles: 0.426
California Keto, Miles: 0.491
Whole Foods Market, Miles: 0.578
Ralphs, Miles: 0.596
Trader Joe's, Miles: 0.614
Tender Greens - Patio Open, Miles: 0.771
News Stand, Miles: 0.777
Williams-Sonoma, Miles: 0.878


In [118]:
result = get_business_radius(32.865028, -117.221756, 1, 'restaurant')
for r in result:
    print('Name: ' + dict(dict(r)['n'])['name'] + ', Category: '+ dict(dict(r)['c'])['name'] + ', Miles: ' + str(np.round(dict(r)['d']/1.6, 3)))

Name: Costa Verde Center, Category: restaurant, Miles: 0.426
Name: Tender Greens - Patio Open, Category: american restaurant, Miles: 0.771
Name: Tender Greens - Patio Open, Category: restaurant, Miles: 0.771


In [116]:
result = get_bus_stops(32.865028, -117.221756,0.5)
for r in result:
    print('Route Id: ' + dict(dict(r)['r'])['rid'] + ', Stop: ' + dict(dict(r)['n'])['name'] + ', Miles: ' + str(np.round(dict(r)['d']/1.6, 3)))

Route Id: 202, Stop: Lebon Dr & Palmilla Dr, Miles: 0.191
Route Id: 201, Stop: Palmilla Dr & Lebon Dr, Miles: 0.204
Route Id: 202, Stop: Regents Rd & Nobel Dr, Miles: 0.207
Route Id: 201, Stop: Nobel Dr & Regents Rd, Miles: 0.239
Route Id: 30, Stop: Nobel Dr & Regents Rd, Miles: 0.239
Route Id: 201, Stop: Regents Rd & Arriba St, Miles: 0.239
Route Id: 30, Stop: Nobel Dr & Regents Rd, Miles: 0.241
Route Id: 202, Stop: Arriba St & Regents Rd, Miles: 0.263
Route Id: 201, Stop: Nobel Dr & Lebon Dr, Miles: 0.303
Route Id: 30, Stop: Nobel Dr & Lebon Dr, Miles: 0.303


In [70]:
result = get_businesses_on_route(202, 0.25)
for r in result:
    print(dict(dict(r)['b'])['name'])

Vons
Costa Verde Center
California Keto
Bristol Farms
Tender Greens - Patio Open
News Stand
Whole Foods Market
Ralphs
Trader Joe's
Target


In [134]:
bbox_5_805_52 = (32.849598, -117.248886, 32.894520, -117.195918)
result = get_poi_bbox(*bbox_5_805_52)
for i, r in enumerate(result):
    if i == 10:
        break
    print(dict(dict(r)['n'])['name'])

UTC Transit Center
UTC Transit Center
UTC Transit Center
Genesee Av & Centurion Sq
Governor Dr & Regents Rd
Judicial Dr & Research Pl
Vons
Genesee Av & Decoro St
UTC Transit Center
UTC Station
